In [35]:

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

pd.set_option('display.max_columns',0)
pd.set_option('display.max_colwidth',0)

match_lvl_data = pd.read_csv('./content/664389efa0868_match_level_scorecard.csv')
batsman_lvl_data = pd.read_csv('./content/663e2b548c98c_batsman_level_scorecard.csv')
bowler_lvl_data = pd.read_csv('./content/663e2b2c60743_bowler_level_scorecard.csv')
train_data = pd.read_csv('./content/663e2b6d54457_train_data_with_samplefeatures.csv')
test_data = pd.read_csv('./content/6644a1e287df6_test_data_with_samplefeatures.csv')



match_lvl_data.shape
match_lvl_data.head(2)

batsman_lvl_data.shape
batsman_lvl_data.head(2)

bowler_lvl_data.shape
bowler_lvl_data.head(2)

train_data.shape
train_data.head(2)

test_data.shape
test_data.head(2)




train_data['winner_01'] = train_data.apply(lambda x: 0 if (x['team1']==x['winner']) else 1, axis=1)


(1689, 30)

,match id,team1,team2,winner,by,win amount,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,umpire1,umpire2,inning1_runs,inning1_wickets,inning1_balls,inning2_runs,inning2_wickets,inning2_balls,team1_id,team1_roster_ids,team2_id,team2_roster_ids,series_type,winner_id,player_of_the_match_id
0,8638034,Nn Ds,Wn,Wn,wickets,9.0,Wn,field,By Ol,Mount Maunganui,2021-01-01,night match,Sr Sh,2020/21,17681,Dn Mw,TJ Pe,148,6,130,152.0,1.0,97.0,17982,7907451.0:4381761.0:31464.0:258649.0:4949790.0:3834305.0:3776849.0:6718340.0:3566240.0:4950364.0:7543647.0,18570,2653993.0:6718326.0:6718382.0:2486896.0:2288789.0:4950224.0:8187878.0:3083667.0:4950294.0:6718396.0:1585464.0,other_domestic,18570,NaN
1,8588005,Me Rs,Sy Tr,Sy Tr,runs,7.0,Sy Tr,field,Ca Ol,Carrara,2021-01-01,night match,Bg Bh Le,2020/21,6663,D Tr,PJ Ge,166,6,106,117.0,2.0,74.0,33942,37351.0:46794.0:5406540.0:2231928.0:181404.0:1594319.0:7534687.0:7537067.0:1905847.0:8339701.0:7620346.0,33963,1506098.0:1749075.0:36665.0:2083409.0:7534652.0:5788320.0:1612610.0:5509524.0:5497274.0:3876613.0:8193310.0,other_domestic,33963,1749075.0


(24483, 21)

,match id,batsman,batsman_id,batsman_details,is_batsman_captain,is_batsman_keeper,inning,runs,balls_faced,over_faced_first,wicket kind,out_by_bowler,out_by_fielder,bowler_id,bowler_details,is_bowler_keeper,is_bowler_captain,strike_rate,Fours,Sixes,match_dt
0,8638034,KD Ce,7907451.0,NZ:Right-hand bat:Right-arm medium-fast:,0.0,0.0,1,7,5,1.1,caught,JS Nm,TM Jn,2486896.0,NZ:Left-hand bat:Right-arm medium-fast:,0.0,0.0,140.0,1.0,NaN,2021-01-01
1,8638034,TL St,4381761.0,NZ:Right-hand bat:None:,0.0,1.0,1,46,46,1.2,caught,R Ra,MG Bl,6718382.0,NZ:Left-hand bat:Slow left-arm orthodox:,0.0,0.0,100.0,4.0,1.0,2021-01-01


(18539, 18)

,match id,bowler,bowler_id,bowler_details,is_bowler_captain,is_bowler_keeper,inning,runs,wicket_count,balls_bowled,economy,maiden,dots,Fours,Sixes,wides,noballs,match_dt
0,8638034,BV Ss,6718396.0,NZ:Right-hand bat:Right-arm medium-fast:,0.0,0.0,1,21,0,12,10.5,0,3.0,1,1,3,1,2021-01-01
1,8638034,HK Bt,1585464.0,NZ:Left-hand bat:Right-arm medium-fast:,0.0,0.0,1,34,1,24,8.5,0,7.0,3,1,2,0,2021-01-01


(948, 23)

,match id,team1,team1_id,team1_roster_ids,team2,team2_id,team2_roster_ids,winner,winner_id,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15
0,9331181,Ba,11283,9373356.0:7857520.0:4232164.0:4566540.0:3299407.0:232000.0:7883504.0:8725723.0:7878989.0:2526390.0:8725730.0:4017523.0,Hl Ph,12634,3500958.0:4231751.0:2735081.0:2035102.0:3698337.0:5058192.0:6844718.0:4566526.0:6596582.0:4161975.0:7952804.0:8465057.0,Hl Ph,12634,Hl Ph,field,Hr Ct Sm Ie,Indore,2022-10-20,day/night match,Sd Mq Ai Ty,2022/23,7398,1.666667,0.672131,139.0,100.0,157.178571
1,8797060,Ed,20,2089079.0:6139370.0:2076192.0:62432.0:2083409.0:172199.0:2022957.0:1482998.0:1711492.0:4507208.0:3231437.0,Wt Is,41,4690258.0:4069666.0:4230127.0:1942317.0:1613926.0:2740408.0:4739552.0:5744850.0:4690104.0:3715697.0:3468870.0,Ed,20,Wt Is,field,Kn Ol Bn Bs,Bridgetown,2022-01-23,day/night match,Ed tr of Wt Is,2021/22,1406,1.285714,1.952381,156.0,50.0,103.500000


(271, 21)

,match id,team1,team1_id,team1_roster_ids,team2,team2_id,team2_roster_ids,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15
0,9250275,Jy,28594,7438955.0:8271969.0:8369661.0:3685247.0:2590251.0:7785497.0:7785553.0:3296264.0:4613622.0:2904796.0:3685289.0,Ud Ss of Aa,90,2436944.0:8117500.0:6703528.0:3010748.0:1613870.0:8934764.0:2721480.0:1632819.0:2789604.0:6489846.0:1565507.0,Ud Ss of Aa,field,Bo Ac Cb,Bulawayo,2022-07-11,day match,Ud Ss of Aa tr of Ze,2022,3226,0.000000,0.019608,NaN,0.00,NaN
1,9262189,Ga An Ws,36084,3715690.0:6818622.0:4069666.0:393014.0:4690188.0:4739552.0:4690328.0:5744850.0:4739580.0:9266102.0:284339.0,Ja Ts,36098,4690258.0:3761246.0:8464385.0:5742470.0:3150720.0:4690104.0:181404.0:1594319.0:5509524.0:4239794.0:2036649.0,Ja Ts,field,Pe Sm Ga,Providence,2022-09-21,night match,Cn Pr Le,2022,13915,0.615385,0.344262,151.285714,66.67,153.5


In [36]:
%matplotlib inline
from matplotlib import pyplot as plt
import re

def createRnP(X_12, feature, N=5, ylim_lb=0.3, ylim_ub=0.7):
    '''
    Rank and Plot of input feature on the input data. The y-axis shows %team1 wins in each bucket.

    Parameters-
    1. X_12: dataset to build the RnP on.
    2. feature: Feature to build RnP of.
    3. N: number of bins on x-axis. Default 5.
    4. ylim_lb: lower bound of y axis on plot.
    5. ylim_ub: upper bound of y axis on plot.

    Output-
    1. Rank and Plot

    Returns- None
    '''
    df = X_12.copy()
    df[f'{feature}_bin'] = df[feature].rank(pct=True)//(1/N)
    df['count'] = 1
    df['team1_win%'] = df['winner_01'].apply(lambda x: 1-x)
    df['team2_win%'] = df['winner_01'].copy()
    df[f'{feature}_min'] = df[feature].copy()
    df[f'{feature}_max'] = df[feature].copy()
    df_g = df.groupby(f'{feature}_bin').agg({'team1_win%':'mean', 'team2_win%':'mean', 'count':'sum', f'{feature}_min':'min',\
                                            f'{feature}_max':'max'}).reset_index()
    N = min(N,df_g.shape[0])
    blue_bar = df_g['team1_win%'].values.tolist()
    ind = np.arange(N)

    plt.figure(figsize=(10,5))
    plt.bar(ind, blue_bar, label='Team 1 win%')
    plt.axhline(y=0.5, color='r', linestyle='--', label='50% win%')
    plt.ylim(ylim_lb, ylim_ub)
    plt.xlabel(f'{feature} Bins')
    plt.ylabel('% Team1 Win')
    plt.title(f'Rank and Plot for {feature}')
    plt.legend()
    plt.show()


In [37]:

batsman_agg = batsman_lvl_data.groupby('match id').agg({
    'runs': ['sum', 'mean', 'std'],
    'balls_faced': ['sum', 'mean', 'std']
}).reset_index()
batsman_agg.columns = ['match id', 'total_runs', 'avg_runs', 'std_runs', 'total_balls_faced', 'avg_balls_faced', 'std_balls_faced']


bowler_agg = bowler_lvl_data.groupby('match id').agg({
    'runs': ['sum', 'mean', 'std'],
    'wicket_count': ['sum', 'mean', 'std'],
    'balls_bowled': ['sum', 'mean', 'std']
}).reset_index()
bowler_agg.columns = ['match id', 'total_runs_conceded', 'avg_runs_conceded', 'std_runs_conceded', 'total_wickets', 'avg_wickets', 'std_wickets', 'total_balls_bowled', 'avg_balls_bowled', 'std_balls_bowled']


train_data = pd.merge(train_data, batsman_agg, how='left', on='match id')
train_data = pd.merge(train_data, bowler_agg, how='left', on='match id')


test_data = pd.merge(test_data, batsman_agg, how='left', on='match id')
test_data = pd.merge(test_data, bowler_agg, how='left', on='match id')


In [46]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


categorical_columns = ['team1', 'team2', 'toss winner', 'toss decision', 'venue', 'city', 'match_dt', 'lighting', 'series_name', 'season']


for col in categorical_columns:
    train_data[col] = train_data[col].astype('category').cat.codes
    test_data[col] = test_data[col].astype('category').cat.codes


X = train_data.drop(['match id', 'winner', 'winner_id', 'winner_01', 'team1_roster_ids', 'team2_roster_ids'], axis=1)
y = train_data['winner_01']


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


model = XGBClassifier()


param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [6, 8],
    'learning_rate': [0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}


grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)


best_model = grid_search.best_estimator_


y_val_pred = best_model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {val_accuracy}")


cv_scores = cross_val_score(best_model, X, y, cv=5, scoring='accuracy')
print(f"Cross-validation Accuracy: {cv_scores.mean()}")


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 1.0],
                         'learning_rate': [0.1, 0.2], 'max_depth': [6, 8],
                         'n_estimators': [100, 200], 'subsample': [0.8, 1.0]},
             scoring='accuracy')

Validation Accuracy: 0.7947368421052632
Cross-validation Accuracy: 0.8280089111668059


In [47]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score


def create_feature_importance_csv(model, X, filename):
    feature_importances = model.feature_importances_
    features = X.columns
    importance_df = pd.DataFrame({
        'feat_id': np.arange(1, len(features) + 1),
        'feat_name': features,
        'model_feat_imp_train': feature_importances
    })
    importance_df['feat_rank_train'] = importance_df['model_feat_imp_train'].rank(ascending=False)
    importance_df['feat_description'] = 'Description not provided'

    importance_df = importance_df.sort_values(by='model_feat_imp_train', ascending=False).reset_index(drop=True)
    importance_df.to_csv(filename, index=False)
    return importance_df

feature_importance_df = create_feature_importance_csv(best_model, X, 'feature_importance.csv')
print("Feature importance CSV created successfully!")

feature_importance_df.head()


Feature importance CSV created successfully!


,feat_id,feat_name,model_feat_imp_train,feat_rank_train,feat_description
0,28,total_wickets,0.148222,1.0,Description not provided
1,31,total_balls_bowled,0.089747,2.0,Description not provided
2,29,avg_wickets,0.085098,3.0,Description not provided
3,22,total_balls_faced,0.047792,4.0,Description not provided
4,25,total_runs_conceded,0.045193,5.0,Description not provided


In [49]:
feature_descriptions = {
    'team_count_50runs_last15': 'Ratio of number of 50s by players in team1 to number of 50s by players in team2 in last 15 games',
    'team_winp_last5': "Ratio of team1's win % to team2's win % in last 5 games",
    'ground_avg_runs_last15': 'Average runs scored in the ground in last 15 games',
    'team1_winp_team2_last15': "Team1's win percentage against Team2 in last 15 games",
    'team1only_avg_runs_last15': "Team1's avg inning runs in last 15 games",
    'toss_winner_01': 'Toss winner to numerical - 1 if team2 wins, else 0',
    'toss_decision_01': 'Toss decision - categorical - 1 if winner bats, 0 otherwise'
}

selected_features = [
    'team2_recent_win_pct', 'team1_recent_win_pct', 'team1_recent_avg_runs',
    'match_dt', 'season', 'team_count_50runs_last15', 'city',
    'team2_recent_avg_runs', 'team2_id', 'team2'
]

def create_feature_importance_csv_with_descriptions(model, X, feature_descriptions, filename):
    feature_importances = model.feature_importances_
    features = X.columns
    importance_df = pd.DataFrame({
        'feat_id': np.arange(1, len(features) + 1),
        'feat_name': features,
        'model_feat_imp_train': feature_importances
    })
    importance_df['feat_rank_train'] = importance_df['model_feat_imp_train'].rank(ascending=False)
    importance_df['feat_description'] = importance_df['feat_name'].map(feature_descriptions)

    importance_df = importance_df.sort_values(by='model_feat_imp_train', ascending=False).reset_index(drop=True)
    importance_df.to_csv(filename, index=False)
    return importance_df

feature_importance_df_with_descriptions = create_feature_importance_csv_with_descriptions(best_model, X, feature_descriptions, 'feature_importance_with_descriptions.csv')
print("Feature importance CSV with descriptions created successfully!")

feature_importance_df_with_descriptions.head()



Feature importance CSV with descriptions created successfully!


,feat_id,feat_name,model_feat_imp_train,feat_rank_train,feat_description
0,28,total_wickets,0.148222,1.0,NaN
1,31,total_balls_bowled,0.089747,2.0,NaN
2,29,avg_wickets,0.085098,3.0,NaN
3,22,total_balls_faced,0.047792,4.0,NaN
4,25,total_runs_conceded,0.045193,5.0,NaN
